In [ ]:
# import statements

In [ ]:
agent = QLearningAgent(state_dim, action_dim, learning_rate, gamma)
env = GameEnvironment()  # Replace with your game environment

num_episodes = 1000
batch_size = 32
epsilon_decay = 0.99

total_rewards = train_agent(agent, env, num_episodes, batch_size, epsilon_decay)

In [ ]:
def train_agent(agent, env, num_episodes, batch_size, epsilon_decay):
    total_rewards = []
    epsilon = 1.0

    for episode in range(num_episodes):
        game_state = env.reset()
        history = []  # Reset the history for each episode
        done = False
        total_reward = 0

        while not done:
            # Choose an action using epsilon-greedy strategy
            action = agent.get_action(game_state, history, epsilon)

            # Take the chosen action and observe the next state, reward, and done flag from the environment
            next_game_state, reward, done = env.step(action)

            # Add the experience to the agent's replay buffer
            agent.add_experience(game_state, history, action, reward, next_game_state, done)

            # Update the agent's Q-network if there are enough experiences in the replay buffer
            if len(agent.replay_buffer) >= batch_size:
                agent.replay_experience(batch_size)

            # Update the game state and history for the next iteration
            game_state = next_game_state
            history.append((game_state, action))

            # Accumulate the total reward
            total_reward += reward

        # Decay epsilon
        epsilon *= epsilon_decay

        # Save the total reward for this episode
        total_rewards.append(total_reward)

        # Print the episode number and total reward
        print(f"Episode {episode + 1}/{num_episodes} - Total Reward: {total_reward}")

    return total_rewards